# Prática independente - SQL Básico II.


In [22]:
query = """SELECT    
    

    FROM 
    orderdetails AS o INNER JOIN products AS p
    
    GROUP BY 
    ORDER BY 
    LIMIT 3;"""

queryResult = pd.read_sql_query(query, db)

queryResult

DatabaseError: Execution failed on sql 'SELECT    
    

    FROM 
    orderdetails AS o INNER JOIN products AS p
    
    GROUP BY 
    ORDER BY 
    LIMIT 3;': near "FROM": syntax error

In [2]:
import pandas as pd
import sqlite3

db = sqlite3.connect('database.db')

#### Vamos executar um script `SQL` com o método [`.executescript()`](https://docs.python.org/3/library/sqlite3.html#sqlite3.Cursor.executescript).

In [3]:
script = 'sample-database-dump.sql'

db.cursor().executescript(open(script).read())

#### Criar uma conexão com nossa base de dados.

In [4]:
conn = sqlite3.connect('database.db')
curs = conn.cursor()
#conn.close()

#### Exercício 1 - Lista funcionários por chefe.
```SQL
JUNTE E COMBINE TODOS AS LINHAS DAS TABELAS
   employees e1,
   employees e2

PEGUE SÓ LINHAS QUE OBEDEÇAM AO CRITÉRIO
   e1.reportsTo = e2.employeeNumber

MOSTRE AS COLUNAS
   e2.firstName||' '||e2.lastName AS boss,
   e2.jobTitle                    AS department,
   e1.employeeNumber              AS ID,
   e1.firstName||' '||e1.lastName AS employee

ORDENE POR
   boss
```

In [21]:
query = """SELECT 
    e2.firstName||' '||e2.lastName AS boss,
    e2.jobTitle                    AS department,
    e1.employeeNumber              AS ID,
    e1.firstName||' '||e1.lastName AS employee
    
    FROM 
    employees AS e1 INNER JOIN employees AS e2
    
    WHERE 
    e1.reportsTo = e2.employeeNumber

    ORDER BY 
    boss
    
    LIMIT 
    3;"""

queryResult = pd.read_sql_query(query, db)

queryResult

,boss,department,ID,employee
0,Anthony Bow,Sales Manager (NA),1165,Leslie Jennings
1,Anthony Bow,Sales Manager (NA),1166,Leslie Thompson
2,Anthony Bow,Sales Manager (NA),1188,Julie Firrelli


#### Exercício 2 - Inspeciona pedidos de compra.

Dados sobre pedidos aparecem nas tabelas `orders` e `orderdetails` e o que relaciona elas é a coluna `orderNumber`. Vamos inspecionar um único pedido.

```SQL
JUNTE E COMBINE TODOS AS LINHAS DAS TABELAS
   orderdetails

PEGUE SÓ LINHAS QUE OBEDEÇAM AO CRITÉRIO
   orderNumber=10103

MOSTRE AS COLUNAS
    orderLineNumber,
    productCode,
    priceEach,
    quantityOrdered,
    priceEach*quantityOrdered as priceTotal
    
ORDENE POR
   orderLineNumber
```

In [8]:

query = """SELECT    
    o.orderLineNumber,
    o.productCode,
    o.priceEach,
    o.quantityOrdered,
    o.priceEach * o.quantityOrdered as priceTotal

    FROM 
    orderdetails AS o 
    
    WHERE 
    o.orderNumber = 10103
    
    ORDER BY
    o.orderLineNumber
    
    LIMIT 3;"""

queryResult = pd.read_sql_query(query, db)

queryResult


,orderLineNumber,productCode,priceEach,quantityOrdered,priceTotal
0,1,S24_2300,107.34,36,3864.24
1,2,S18_2432,58.34,22,1283.48
2,3,S32_1268,92.46,31,2866.26


#### Exercício 3 - Vamos melhorar a leitura dessa tabela adicionando a descrição do produto. Faremos um **JOIN** com a tabela `products`.

```SQL
JUNTE E COMBINE TODOS AS LINHAS DAS TABELAS
   orderdetails AS o,
   products AS p

PEGUE SÓ LINHAS QUE OBEDEÇAM AO CRITÉRIO
    o.productCode=p.productCode AND
    orderNumber=10103

MOSTRE AS COLUNAS
    o.orderLineNumber,
    o.priceEach,
    o.quantityOrdered,
    o.priceEach*o.quantityOrdered as itemTotal,
    o.productCode,
    p.productName
    
ORDENE POR
   orderLineNumber
```

In [9]:
query = """SELECT    
    o.orderLineNumber,
    o.priceEach,
    o.quantityOrdered,
    o.priceEach*o.quantityOrdered as itemTotal,
    o.productCode,
    p.productName

    FROM 
    orderdetails AS o INNER JOIN products AS p

    WHERE 
    o.productCode = p.productCode AND
    orderNumber = 10103

    LIMIT 3;"""

queryResult = pd.read_sql_query(query, db)

queryResult

,orderLineNumber,priceEach,quantityOrdered,itemTotal,productCode,productName
0,11,214.30,26,5571.80,S10_1949,1952 Alpine Renault 1300
1,4,119.67,42,5026.14,S10_4962,1962 LanciaA Delta 16V
2,8,121.64,27,3284.28,S12_1666,1958 Setra Bus


#### Exercício 4 - Performance de venda por categoria de produtos

```SQL
JUNTE E COMBINE TODOS AS LINHAS DAS TABELAS
    orderdetails o,
    products p

PEGUE SÓ LINHAS QUE OBEDEÇAM AO CRITÉRIO
    o.productCode=p.productCode

MOSTRE AS COLUNAS
    p.productLine,
    count(p.productLine) AS nItems,
    sum(o.priceEach*o.quantityOrdered)/count(p.productLine) AS averagePerItem,
    sum(o.priceEach*o.quantityOrdered) AS lineTotal

AGRUPE POR
    p.productLine

ORDENE INVERSAMENTE POR
   lineTotal
```

In [10]:
query = """SELECT    
    p.productLine,
    count(p.productLine) AS nItems,
    sum(o.priceEach*o.quantityOrdered)/count(p.productLine) AS averagePerItem,
    sum(o.priceEach*o.quantityOrdered) AS lineTotal

    FROM 
    orderdetails o INNER JOIN products p
    
    WHERE
    o.productCode = p.productCode
    
    GROUP BY 
    p.productLine
    
    ORDER BY 
    lineTotal
    
    LIMIT 3;"""

queryResult = pd.read_sql_query(query, db)

queryResult


,productLine,nItems,averagePerItem,lineTotal
0,Trains,81,2327.566914,188532.92
1,Ships,245,2710.197306,663998.34
2,Planes,336,2841.183155,954637.54


#### Exercício 5 - Qual é o produto que mais vende ?

```SQL
JUNTE E COMBINE TODOS AS LINHAS DAS TABELAS
    orderdetails o,
    products p

PEGUE SÓ LINHAS QUE OBEDEÇAM AO CRITÉRIO
    o.productCode=p.productCode

MOSTRE AS COLUNAS
    p.productName,
    count(o.quantityOrdered) AS nItems,
#    sum(o.priceEach*o.quantityOrdered) AS lineTotal

AGRUPE POR
    p.productCode

ORDENE INVERSAMENTE POR
   lineTotal
```

In [12]:

query = """SELECT    
    p.productName,
    count(o.quantityOrdered) AS nItems,
    sum(o.priceEach * o.quantityOrdered) AS lineTotal


    FROM 
    orderdetails AS o INNER JOIN products AS p

    WHERE
    o.productCode = p.productCode
    
    LIMIT 3;"""

queryResult = pd.read_sql_query(query, db)

queryResult

,productName,nItems,lineTotal
0,1917 Grand Touring Sedan,2996,9604190.61


#### Exercício 6 - Qual é o cliente que mais compra ?

```SQL
JUNTE E COMBINE TODOS AS LINHAS DAS TABELAS
    orderdetails as od,
    orders as o,
    customers as c

PEGUE SÓ LINHAS QUE OBEDEÇAM AO CRITÉRIO
    od.orderNumber = o.orderNumber AND
    c.customerNumber = o.customerNumber

MOSTRE AS COLUNAS
    c.customerNumber,
    c.customerName,
    sum(od.priceEach*od.quantityOrdered) AS customerTotal
    sum(od.quantityOrdered) AS nItems,

AGRUPE POR
    c.customerNumber

ORDENE INVERSAMENTE POR
   customerTotal
```

In [18]:
query = """SELECT    
    c.customerNumber,
    c.customerName,
    sum(od.priceEach * od.quantityOrdered) AS customerTotal,
    sum(od.quantityOrdered) AS nItems

    FROM 
    orderdetails AS od INNER JOIN orders AS o INNER JOIN customers AS c
    
    WHERE 
    od.orderNumber = o.orderNumber AND c.customerNumber = o.customerNumber
    
    GROUP BY
    c.customerNumber
    
    ORDER BY 
    customerTotal
    LIMIT 3;"""

queryResult = pd.read_sql_query(query, db)

queryResult

,customerNumber,customerName,customerTotal,nItems
0,219,Boards & Toys Co.,7918.60,102
1,198,Auto-Moto Classics Inc.,21554.26,287
2,103,Atelier graphique,22314.36,270


#### Exercício 7 - Mostre vendas e valores por mês...

```SQL
JUNTE E COMBINE TODOS AS LINHAS DAS TABELAS
    orderdetails as od,
    orders as o,

PEGUE SÓ LINHAS QUE OBEDEÇAM AO CRITÉRIO
    od.orderNumber = o.orderNumber

MOSTRE AS COLUNAS
    substr(o.orderDate,1,7) as month,
    sum(od.priceEach*od.quantityOrdered) AS monthTotal
    sum(od.quantityOrdered) AS nItems,

AGRUPE POR
    month

ORDENE INVERSAMENTE POR
   month
```

In [20]:
query = """SELECT    
    substr(o.orderDate,1,7) as month,
    sum(od.priceEach*od.quantityOrdered) AS monthTotal,
    sum(od.quantityOrdered) AS nItems


    FROM 
    orderdetails AS od INNER JOIN orders AS o
    
    WHERE
    od.orderNumber = o.orderNumber

    GROUP BY 
    month
    
    ORDER BY 
    month
    
    LIMIT 3;"""

queryResult = pd.read_sql_query(query, db)

queryResult

,month,monthTotal,nItems
0,2003-01,116692.77,1357
1,2003-02,128403.64,1449
2,2003-03,160517.14,1755
